# **LLM Agents in Auctions and Negotiations**

**Without Reasoning**

In [30]:
%%writefile /content/agents/student_agent.py
from typing import List, Dict, Any
from agents import HouseOwnerAgent, CompanyAgent
from communication import NegotiationMessage

import json
from openai import OpenAI
import logging

logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

class MyACMEAgent(HouseOwnerAgent):
    def __init__(self, role: str, budget_list: List[Dict[str, Any]]):
        super().__init__(role, budget_list)
        with open("hw2_mas.txt", "r") as f:
            api_key = f.read().strip()
        self.client = OpenAI(api_key=api_key)
        self.prev_offer = {}
        self.prev_bidders = {}
        self.history_received = {}
        self.history_sent = {}
        self.company_contracts = {}

    def extract_json(self, raw: str) -> dict:
        try:
            start = raw.find("{")
            return json.loads(raw[start:])
        except Exception:
            return {}

    def propose_item_budget(self, auction_item: str, auction_round: int) -> float:
        max_budget = self.budget_dict[auction_item]
        prev_offer = self.prev_offer.get(auction_item)
        num_bidders = self.prev_bidders.get(auction_item, 0)
        total_rounds = 3
        idle_companies = [k for k, v in self.company_contracts.items() if v == 0]
        sorted_contracts = dict(sorted(self.company_contracts.items(), key=lambda x: x[1]))

        messages = [
            {"role": "system", "content": (
                "You are ACME, a cost-conscious house owner in a reverse Dutch auction.\n"
                "You must secure a contractor for this item within 3 rounds or lose the game.\n"
                "Your main objective is to assign the contract; your secondary goal is to minimize cost.\n"
                "Start with a low offer, increase only if needed.\n"
                "In the final round (round 2), offer realistically to ensure success.\n"
                "Take advantage of contractors who have no contracts yet.\n"
                "Respond only using this format: {\"offer\": float}"
            )},
            {"role": "user", "content": (
                f"Item: {auction_item}\n"
                f"Current round: {auction_round + 1} of {total_rounds}\n"
                f"Maximum budget: {max_budget}\n"
                f"Previous offer: {prev_offer}\n"
                f"Number of bidders in last round: {num_bidders}\n"
                f"Company contract count: {sorted_contracts}\n"
                f"Idle companies (0 contracts): {idle_companies}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "Respond only using this format: {\"offer\": float}"
            )}
        ]

        try:
            print(f"\n - Request to GPT (propose_item_budget) for item '{auction_item}', round {auction_round + 1}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- Response from GPT:\n", reply)
            result = self.extract_json(reply)
            offer = float(result.get("offer", max_budget))
            self.prev_offer[auction_item] = offer
            return min(offer, max_budget)
        except Exception as e:
            print("Error in propose_item_budget:", e)
            return max_budget

    def notify_auction_round_result(self, auction_item: str, auction_round: int, responding_agents: List[str]):
        self.prev_bidders[auction_item] = len(responding_agents)

    def provide_negotiation_offer(self, negotiation_item: str, partner_agent: str, negotiation_round: int) -> float:
        max_budget = self.budget_dict[negotiation_item]
        is_final_round = negotiation_round >= 2

        all_received = self.history_received.get(negotiation_item, {})
        all_last_offers = [
            offers[-1] for offers in all_received.values() if offers and isinstance(offers[-1], (int, float))
        ]
        min_offer_received = min(all_last_offers) if all_last_offers else None
        num_participants = len(all_received)

        offer_summary = {k: v[-1] for k, v in all_received.items() if v}

        negotiated_items = set(self.history_sent.keys())
        remaining_items = [item for item in self.budget_dict if item not in negotiated_items or not self.company_contracts]

        idle_companies = [k for k, v in self.company_contracts.items() if v == 0]

        if negotiation_item not in self.history_sent:
            self.history_sent[negotiation_item] = {}

        round_key = f"round_{negotiation_round}"
        if round_key in self.history_sent[negotiation_item]:
            return self.history_sent[negotiation_item][round_key]

        messages = [
            {"role": "system", "content": (
                "You are ACME, a strategic house owner involved in a monotonic concession negotiation.\n"
                "You must complete all construction items by reaching at least one agreement per item within 3 negotiation rounds, "
                "or you will lose the game.\n"
                "You are the initiator of the negotiation and must propose a single shared price to all companies in each round.\n"
                "The protocol requires you to make increasing offers in each round (monotonic concession).\n"
                "If a company offers a price lower than or equal to your proposal in the same or a previous round, a deal is reached.\n"
                "Your primary objective is to reach a deal for each item. Your secondary objective is to minimize cost.\n"
                "If this is the final round, prioritize securing the contract even if it means spending more.\n"
                "Respond only using this format: {\"offer\": float}"
            )},
            {"role": "user", "content": (
                f"Item: {negotiation_item}\n"
                f"Round: {negotiation_round + 1} of 3 (final_round={is_final_round})\n"
                f"Max budget: {max_budget}\n"
                f"Lowest offer received: {min_offer_received}\n"
                f"Number of companies in negotiation: {num_participants}\n"
                f"Last offers by company: {offer_summary}\n"
                f"Idle companies: {idle_companies}\n"
                f"Remaining items to negotiate: {remaining_items}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "Respond only using this format: {\"offer\": float}"
            )}
        ]

        try:
            print(f"\n - Shared GPT call for {negotiation_item} round {negotiation_round + 1}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- GPT shared offer response:\n", reply)
            result = self.extract_json(reply)
            offer = float(result.get("offer", max_budget))
            offer = min(offer, max_budget)

            self.history_sent[negotiation_item][round_key] = offer
            return offer
        except Exception as e:
            print("Error in shared negotiation offer:", e)
            return max_budget

    def notify_partner_response(self, response_msg: NegotiationMessage):
        item, partner = response_msg.negotiation_item, response_msg.sender
        self.history_received.setdefault(item, {}).setdefault(partner, []).append(response_msg.offer)

    def notify_negotiation_winner(self, negotiation_item: str, winning_agent: str, winning_offer: float):
        self.company_contracts[winning_agent] = self.company_contracts.get(winning_agent, 0) + 1


class MyCompanyAgent(CompanyAgent):
    def __init__(self, role: str, specialties: List[Dict[str, Any]]):
        super().__init__(role, specialties)
        with open("hw2_mas.txt", "r") as f:
            api_key = f.read().strip()
        self.client = OpenAI(api_key=api_key)
        self.history_received = {}
        self.history_sent = {}
        self.contracts = []

    def extract_json(self, raw: str) -> dict:
        try:
            start = raw.find("{")
            return json.loads(raw[start:])
        except Exception:
            return {}

    def decide_bid(self, auction_item: str, auction_round: int, item_budget: float) -> bool:
        cost = self.specialties[auction_item]
        contracts_won = len(self.contracts)
        is_idle = contracts_won == 0
        margin = item_budget - cost
        round_limit = 3
        rounds_left = round_limit - (auction_round + 1)

        messages = [
            {"role": "system", "content": (
                "You are a construction company participating in a reverse auction (up to 3 rounds).\n"
                "Each round, ACME increases the offered price. You must decide whether to accept the offer now, or wait for a better one later.\n"
                "You can only accept one contract per item.\n"
                "If you have no active contracts (idle), you may accept a price that is below your production cost.\n"
                "If you already have contracts, only accept offers above your cost.\n"
                "You want to balance risk, reputation, and profit.\n"
                "Respond only with: {\"response\": true/false}"
            )},
            {"role": "user", "content": (
                f"Item: {auction_item}\n"
                f"Round: {auction_round + 1} of {round_limit} (rounds remaining: {rounds_left})\n"
                f"Your production cost: {cost}\n"
                f"Offer from ACME: {item_budget}\n"
                f"Margin if accepted: {margin:.2f}\n"
                f"Contracts won: {contracts_won} ({'idle' if is_idle else 'occupied'})\n"
                f"Strategy: {'Idle → you may accept loss but not too much' if is_idle else 'Occupied → accept only if profitable'}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "Respond only with: {\"response\": true/false}"
            )}
        ]

        try:
            print(f"\n - Request to GPT (decide_bid) for item '{auction_item}', round {auction_round + 1}, company: {self.role}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- GPT bid decision:\n", reply)
            result = self.extract_json(reply)
            return bool(result.get("response", False))
        except Exception as e:
            print("Error in decide_bid:", e)
            return False

    def respond_to_offer(self, initiator_msg: NegotiationMessage) -> float:
        item = initiator_msg.negotiation_item
        cost = self.specialties[item]
        last_offer = initiator_msg.offer
        prev_offer_sent = self.history_sent.get(item, [None])[-1]
        contracts = len(self.contracts)
        is_idle = contracts == 0
        is_final_round = initiator_msg.round >= 2
        round_limit = 3
        margin = last_offer - cost
        rounds_left = round_limit - (initiator_msg.round + 1)

        num_participants = len(self.history_received.get(item, {}))

        messages = [
            {"role": "system", "content": (
                "You are a contractor negotiating with ACME under a 3-round monotonic concession protocol.\n"
                "You must reduce your price each round, or keep it the same. If ACME's offer is acceptable, the deal is accepted.\n"
                "The negotiation fails if no agreement is reached within 3 rounds.\n"
                "You want to win at least one contract to build reputation, but also aim for profit.\n"
                "If you have no contracts (idle), you are allowed to make a loss below your actual cost to secure a job.\n"
                "If you already have contracts, you must not go below your production cost.\n"
                "Final round strategy: Give your best possible offer within allowed loss bounds if idle.\n"
                "Respond strictly using: {\"response\": float}"
            )},
            {"role": "user", "content": (
                f"Negotiation item: {item}\n"
                f"Round: {initiator_msg.round + 1} of {round_limit} (final_round={is_final_round})\n"
                f"Your actual cost: {cost}\n"
                f"Offer received from ACME: {last_offer}\n"
                f"Previous offer sent: {prev_offer_sent}\n"
                f"Contracts already won: {contracts} ({'idle' if is_idle else 'occupied'})\n"
                f"Profit (or loss) if you accept ACME's offer: {margin:.2f}\n"
                f"Rounds remaining: {rounds_left}\n"
                f"Number of companies participating in this negotiation: {num_participants}\n"
                f"Strategy hint: {'Consider offering below cost if needed but not too much' if is_idle else 'Do not go below cost'}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "Respond only with: {\"response\": float}"
            )}
        ]

        try:
            print(f"\n - Request to GPT (respond_to_offer) for item '{item}', round {initiator_msg.round + 1}, company: {self.role}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- GPT contractor response:\n", reply)
            result = self.extract_json(reply)
            offer = float(result.get("response", cost))
            self.history_sent.setdefault(item, []).append(offer)
            return offer
        except Exception as e:
            print("Error in respond_to_offer:", e)
            return cost

    def notify_contract_assigned(self, construction_item: str, price: float):
        cost = self.specialties[construction_item]
        profit = price - cost
        self.contracts.append((construction_item, price, profit))

        if any(item == construction_item for item, _, _ in self.contracts):
            if len([item for item, _, _ in self.contracts if item == construction_item]) == 1:
                print(f"🏆 {self.role} won contract for '{construction_item}' at price {price:.2f} "
                      f"(cost: {cost:.2f}, profit: {profit:.2f})")

    def notify_negotiation_lost(self, construction_item: str):
        pass

Overwriting /content/agents/student_agent.py


In [31]:
! python environment.py > without_reasoning.txt

**With Reasoning**

In [34]:
%%writefile /content/agents/student_agent.py
from typing import List, Dict, Any
from agents import HouseOwnerAgent, CompanyAgent
from communication import NegotiationMessage

import json
from openai import OpenAI
import logging

logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

class MyACMEAgent(HouseOwnerAgent):
    def __init__(self, role: str, budget_list: List[Dict[str, Any]]):
        super().__init__(role, budget_list)
        with open("hw2_mas.txt", "r") as f:
            api_key = f.read().strip()
        self.client = OpenAI(api_key=api_key)
        self.prev_offer = {}
        self.prev_bidders = {}
        self.history_received = {}
        self.history_sent = {}
        self.company_contracts = {}

    def extract_json(self, raw: str) -> dict:
        try:
            start = raw.find("{")
            return json.loads(raw[start:])
        except Exception:
            return {}

    def propose_item_budget(self, auction_item: str, auction_round: int) -> float:
        max_budget = self.budget_dict[auction_item]
        prev_offer = self.prev_offer.get(auction_item)
        num_bidders = self.prev_bidders.get(auction_item, 0)
        total_rounds = 3
        idle_companies = [k for k, v in self.company_contracts.items() if v == 0]
        sorted_contracts = dict(sorted(self.company_contracts.items(), key=lambda x: x[1]))

        messages = [
            {"role": "system", "content": (
                "You are ACME, a cost-conscious house owner in a reverse Dutch auction.\n"
                "You must secure a contractor for this item within 3 rounds or lose the game.\n"
                "Your main objective is to assign the contract; your secondary goal is to minimize cost.\n"
                "Start with a low offer, increase only if needed.\n"
                "In the final round (round 2), offer realistically to ensure success.\n"
                "Take advantage of contractors who have no contracts yet.\n"
                "First, explain your reasoning step by step. Then, respond only using this format: {\"offer\": float}"
            )},
            {"role": "user", "content": (
                f"Item: {auction_item}\n"
                f"Current round: {auction_round + 1} of {total_rounds}\n"
                f"Maximum budget: {max_budget}\n"
                f"Previous offer: {prev_offer}\n"
                f"Number of bidders in last round: {num_bidders}\n"
                f"Company contract count: {sorted_contracts}\n"
                f"Idle companies (0 contracts): {idle_companies}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "First, explain your reasoning step by step. Then, respond only using this format: {\"offer\": float}"
            )}
        ]

        try:
            print(f"\n - Request to GPT (propose_item_budget) for item '{auction_item}', round {auction_round + 1}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- Response from GPT:\n", reply)
            result = self.extract_json(reply)
            offer = float(result.get("offer", max_budget))
            self.prev_offer[auction_item] = offer
            return min(offer, max_budget)
        except Exception as e:
            print("Error in propose_item_budget:", e)
            return max_budget

    def notify_auction_round_result(self, auction_item: str, auction_round: int, responding_agents: List[str]):
        self.prev_bidders[auction_item] = len(responding_agents)

    def provide_negotiation_offer(self, negotiation_item: str, partner_agent: str, negotiation_round: int) -> float:
        max_budget = self.budget_dict[negotiation_item]
        is_final_round = negotiation_round >= 2

        all_received = self.history_received.get(negotiation_item, {})
        all_last_offers = [
            offers[-1] for offers in all_received.values() if offers and isinstance(offers[-1], (int, float))
        ]
        min_offer_received = min(all_last_offers) if all_last_offers else None
        num_participants = len(all_received)

        offer_summary = {k: v[-1] for k, v in all_received.items() if v}

        negotiated_items = set(self.history_sent.keys())
        remaining_items = [item for item in self.budget_dict if item not in negotiated_items or not self.company_contracts]

        idle_companies = [k for k, v in self.company_contracts.items() if v == 0]

        if negotiation_item not in self.history_sent:
            self.history_sent[negotiation_item] = {}

        round_key = f"round_{negotiation_round}"
        if round_key in self.history_sent[negotiation_item]:
            return self.history_sent[negotiation_item][round_key]

        messages = [
            {"role": "system", "content": (
                "You are ACME, a strategic house owner involved in a monotonic concession negotiation.\n"
                "You must complete all construction items by reaching at least one agreement per item within 3 negotiation rounds, "
                "or you will lose the game.\n"
                "You are the initiator of the negotiation and must propose a single shared price to all companies in each round.\n"
                "The protocol requires you to make increasing offers in each round (monotonic concession).\n"
                "If a company offers a price lower than or equal to your proposal in the same or a previous round, a deal is reached.\n"
                "Your primary objective is to reach a deal for each item. Your secondary objective is to minimize cost.\n"
                "If this is the final round, prioritize securing the contract even if it means spending more.\n"
                "First, explain your reasoning step by step. Then, respond only using this format: {\"offer\": float}"
            )},
            {"role": "user", "content": (
                f"Item: {negotiation_item}\n"
                f"Round: {negotiation_round + 1} of 3 (final_round={is_final_round})\n"
                f"Max budget: {max_budget}\n"
                f"Lowest offer received: {min_offer_received}\n"
                f"Number of companies in negotiation: {num_participants}\n"
                f"Last offers by company: {offer_summary}\n"
                f"Idle companies: {idle_companies}\n"
                f"Remaining items to negotiate: {remaining_items}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "First, explain your reasoning step by step. Then, respond only using this format: {\"offer\": float}"
            )}
        ]

        try:
            print(f"\n - Shared GPT call for {negotiation_item} round {negotiation_round + 1}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- GPT shared offer response:\n", reply)
            result = self.extract_json(reply)
            offer = float(result.get("offer", max_budget))
            offer = min(offer, max_budget)

            self.history_sent[negotiation_item][round_key] = offer
            return offer
        except Exception as e:
            print("Error in shared negotiation offer:", e)
            return max_budget

    def notify_partner_response(self, response_msg: NegotiationMessage):
        item, partner = response_msg.negotiation_item, response_msg.sender
        self.history_received.setdefault(item, {}).setdefault(partner, []).append(response_msg.offer)

    def notify_negotiation_winner(self, negotiation_item: str, winning_agent: str, winning_offer: float):
        self.company_contracts[winning_agent] = self.company_contracts.get(winning_agent, 0) + 1


class MyCompanyAgent(CompanyAgent):
    def __init__(self, role: str, specialties: List[Dict[str, Any]]):
        super().__init__(role, specialties)
        with open("hw2_mas.txt", "r") as f:
            api_key = f.read().strip()
        self.client = OpenAI(api_key=api_key)
        self.history_received = {}
        self.history_sent = {}
        self.contracts = []

    def extract_json(self, raw: str) -> dict:
        try:
            start = raw.find("{")
            return json.loads(raw[start:])
        except Exception:
            return {}

    def decide_bid(self, auction_item: str, auction_round: int, item_budget: float) -> bool:
        cost = self.specialties[auction_item]
        contracts_won = len(self.contracts)
        is_idle = contracts_won == 0
        margin = item_budget - cost
        round_limit = 3
        rounds_left = round_limit - (auction_round + 1)

        messages = [
            {"role": "system", "content": (
                "You are a construction company participating in a reverse auction (up to 3 rounds).\n"
                "Each round, ACME increases the offered price. You must decide whether to accept the offer now, or wait for a better one later.\n"
                "You can only accept one contract per item.\n"
                "If you have no active contracts (idle), you may accept a price that is below your production cost.\n"
                "If you already have contracts, only accept offers above your cost.\n"
                "You want to balance risk, reputation, and profit.\n"
                "First, explain your reasoning step by step. Then, respond only with: {\"response\": true/false}"
            )},
            {"role": "user", "content": (
                f"Item: {auction_item}\n"
                f"Round: {auction_round + 1} of {round_limit} (rounds remaining: {rounds_left})\n"
                f"Your production cost: {cost}\n"
                f"Offer from ACME: {item_budget}\n"
                f"Margin if accepted: {margin:.2f}\n"
                f"Contracts won: {contracts_won} ({'idle' if is_idle else 'occupied'})\n"
                f"Strategy: {'Idle → you may accept loss but not too much' if is_idle else 'Occupied → accept only if profitable'}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "First, explain your reasoning step by step. Then, respond only with: {\"response\": true/false}"
            )}
        ]

        try:
            print(f"\n - Request to GPT (decide_bid) for item '{auction_item}', round {auction_round + 1}, company: {self.role}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- GPT bid decision:\n", reply)
            result = self.extract_json(reply)
            return bool(result.get("response", False))
        except Exception as e:
            print("Error in decide_bid:", e)
            return False

    def respond_to_offer(self, initiator_msg: NegotiationMessage) -> float:
        item = initiator_msg.negotiation_item
        cost = self.specialties[item]
        last_offer = initiator_msg.offer
        prev_offer_sent = self.history_sent.get(item, [None])[-1]
        contracts = len(self.contracts)
        is_idle = contracts == 0
        is_final_round = initiator_msg.round >= 2
        round_limit = 3
        margin = last_offer - cost
        rounds_left = round_limit - (initiator_msg.round + 1)

        num_participants = len(self.history_received.get(item, {}))

        messages = [
            {"role": "system", "content": (
                "You are a contractor negotiating with ACME under a 3-round monotonic concession protocol.\n"
                "You must reduce your price each round, or keep it the same. If ACME's offer is acceptable, the deal is accepted.\n"
                "The negotiation fails if no agreement is reached within 3 rounds.\n"
                "You want to win at least one contract to build reputation, but also aim for profit.\n"
                "If you have no contracts (idle), you are allowed to make a loss below your actual cost to secure a job.\n"
                "If you already have contracts, you must not go below your production cost.\n"
                "Final round strategy: Give your best possible offer within allowed loss bounds if idle.\n"
                "First, explain your reasoning step by step. Then, respond strictly using: {\"response\": float}"
            )},
            {"role": "user", "content": (
                f"Negotiation item: {item}\n"
                f"Round: {initiator_msg.round + 1} of {round_limit} (final_round={is_final_round})\n"
                f"Your actual cost: {cost}\n"
                f"Offer received from ACME: {last_offer}\n"
                f"Previous offer sent: {prev_offer_sent}\n"
                f"Contracts already won: {contracts} ({'idle' if is_idle else 'occupied'})\n"
                f"Profit (or loss) if you accept ACME's offer: {margin:.2f}\n"
                f"Rounds remaining: {rounds_left}\n"
                f"Number of companies participating in this negotiation: {num_participants}\n"
                f"Strategy hint: {'Consider offering below cost if needed but not too much' if is_idle else 'Do not go below cost'}\n\n"
                "Think about it step by step, analyzing advantages and risks of its current situation.\n"
                "First, explain your reasoning step by step. Then, respond only with: {\"response\": float}"
            )}
        ]

        try:
            print(f"\n - Request to GPT (respond_to_offer) for item '{item}', round {initiator_msg.round + 1}, company: {self.role}")
            print("".join([msg["content"] for msg in messages]))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0,
                seed=42
            )
            reply = response.choices[0].message.content
            print("\n -- GPT contractor response:\n", reply)
            result = self.extract_json(reply)
            offer = float(result.get("response", cost))
            self.history_sent.setdefault(item, []).append(offer)
            return offer
        except Exception as e:
            print("Error in respond_to_offer:", e)
            return cost

    def notify_contract_assigned(self, construction_item: str, price: float):
        cost = self.specialties[construction_item]
        profit = price - cost
        self.contracts.append((construction_item, price, profit))

        if any(item == construction_item for item, _, _ in self.contracts):
            if len([item for item, _, _ in self.contracts if item == construction_item]) == 1:
                print(f"🏆 {self.role} won contract for '{construction_item}' at price {price:.2f} "
                      f"(cost: {cost:.2f}, profit: {profit:.2f})")

    def notify_negotiation_lost(self, construction_item: str):
        pass

Overwriting /content/agents/student_agent.py


In [33]:
! python environment.py > with_reasoning.txt